In [1]:
!pip install transformers
!pip install sentencepiece
import tensorflow as tf
import numpy as np
import pandas as pd
from transformers import *
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
import re

     |████████████████████████████████| 2.9 MB 5.3 MB/s 
     |████████████████████████████████| 56 kB 4.8 MB/s 
     |████████████████████████████████| 895 kB 33.6 MB/s 
     |████████████████████████████████| 3.3 MB 48.8 MB/s 
     |████████████████████████████████| 596 kB 42.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.2 MB 5.3 MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [3]:
data = pd.read_csv('/content/educate_data_eng.csv')

In [4]:
data.head()

,Unnamed: 0,lyr,0,label
0,0,Aye aye aye Red rubies and they bleed Pop some...,sexy,3.0
1,1,Rap La la la la la Rap Boy I wanna be with you...,sad,0.0
2,3,Oh woah ooh Oh oh eh Verse Chris Brown AGNEZ...,sexy,3.0
3,5,I betcha that I could read your mind And tell ...,sad,0.0
4,6,Baby I been thinkin bout you all the time Late...,sexy,3.0


In [6]:
data.drop('0', axis = 1, inplace = True)

In [7]:
data.columns=['index','data','category']

In [8]:
data = data[['index','category','data']]

In [9]:
data['category'] = pd.to_numeric(data['category'])
data['index'] = pd.to_numeric(data['index'])

In [10]:
data['category'] = data['category'].astype('int')
data['index'] = data['index'].astype('int')

In [11]:
train = data

In [48]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data,test_size=0.2,random_state=7) 

In [12]:
#submission = pd.read_csv('/content/dataEng.csv')

In [13]:
#submission.head()

,index,Unnamed: 0,issue_date,album_name,song_name,song_gn_gnr_basket,artist_name_basket,id,lyric,lyr
0,14,14,20180720,Head In The Clouds,Red Rubies,GN1300,"88rising , Rich Brian , Yung Bans , Yung ...",245,"Aye, aye, aye Red rubies and they bleed Pop so...",Aye aye aye Red rubies and they bleed Pop some...
1,31,31,20020101,Lickin` On Both Sides,All I Want,GN1300,Mis-Teeq,409,(Rap) La la la la la (Rap) Boy I wanna be with...,Rap La la la la la Rap Boy I wanna be with you...
2,35,35,19710000,Live At Filmore West,Make It With You,GN1300,Aretha Franklin,465,Hey have you ever tried Really reaching out fo...,Hey have you ever tried Really reaching out fo...
3,46,46,20180803,Overdose (Feat. Chris Brown),Overdose (Feat. Chris Brown),GN1300,Agnez Mo,651,"Oh woah, ooh Oh, oh, eh Verse 1: Chris Brown,...",Oh woah ooh Oh oh eh Verse Chris Brown AGNEZ...
4,47,47,20140429,스타로부터 스무 발자국 OST,Desperation,GN1300,Judith Hill,669,Maybe you're where you're supposed to be Safel...,Maybe youre where youre supposed to be Safely ...


In [14]:
#test = submission[['index','lyric']]

In [15]:
#test.columns=['index', 'data']

In [49]:
train = train.copy()
test = test.copy()

In [50]:
train['data'] = train['data'].str.replace(r'[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》\\n\t]+', " ", regex=True)
test['data'] = test['data'].str.replace(r'[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', " ", regex=True)
train['data'] = train['data'].str.replace(r'\t+', " ", regex=True)
test['data'] = test['data'].str.replace(r'\t+', " ", regex=True)
train['data'] = train['data'].str.replace(r'[\\n]+'," ", regex=True)
test['data'] = test['data'].str.replace(r'[\\n]+'," ", regex=True)

In [51]:
train = train.reset_index()
test = test.reset_index()

train.drop('level_0', axis= 1, inplace=True)
test.drop('level_0', axis= 1, inplace=True)

In [19]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [63]:
def convert_data(data_df):
    global tokenizer
    
    SEQ_LEN = 256 #SEQ_LEN : 버트에 들어갈 인풋의 길이
    
    tokens, masks, segments, targets = [], [], [], []
    
    for i in tqdm(range(len(data_df))):
        # token : 문장을 토큰화함
        token = tokenizer.encode(data_df[DATA_COLUMN][i], truncation=True, padding='max_length', max_length=SEQ_LEN)
       
        # 마스크는 토큰화한 문장에서 패딩이 아닌 부분은 1, 패딩인 부분은 0으로 통일
        num_zeros = token.count(0)
        mask = [1]*(SEQ_LEN-num_zeros) + [0]*num_zeros
        
        # 문장의 전후관계를 구분해주는 세그먼트는 문장이 1개밖에 없으므로 모두 0
        segment = [0]*SEQ_LEN

        # 버트 인풋으로 들어가는 token, mask, segment를 tokens, segments에 각각 저장
        tokens.append(token)
        masks.append(mask)
        segments.append(segment)
        
        # 정답(긍정 : 1 부정 0)을 targets 변수에 저장해 줌
        targets.append(data_df[LABEL_COLUMN][i])

    # tokens, masks, segments, 정답 변수 targets를 numpy array로 지정    
    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)
    targets = np.array(targets)

    return [tokens, masks, segments], targets

# 위에 정의한 convert_data 함수를 불러오는 함수를 정의
def load_data(pandas_dataframe):
    data_df = pandas_dataframe
    data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)
    data_df[LABEL_COLUMN] = data_df[LABEL_COLUMN].astype(int)
    data_x, data_y = convert_data(data_df)
    return data_x, data_y

SEQ_LEN = 256
BATCH_SIZE = 32
# 긍부정 문장을 포함하고 있는 칼럼
DATA_COLUMN = "data"
# 긍정인지 부정인지를 (1=긍정,0=부정) 포함하고 있는 칼럼
LABEL_COLUMN = "category"

# train 데이터를 버트 인풋에 맞게 변환
train_x, train_y = load_data(train)

100%|██████████| 3970/3970 [00:35<00:00, 111.27it/s]


In [64]:
train

,index,category,data
0,3538,0,Yeah yeah Woke up rich real bad bitch Playboy ...
1,4683,3,Chorus Now what am I supposed to do Whe I wa...
2,4589,0,said you a d i weve bee together for a mi ute...
3,5368,0,Whe Im dow You show me love Whe times get r...
4,5438,1,Soft heave ly eyes gazed i to me Tra sce di g ...
...,...,...,...
3965,6044,3,Everybody gather rou d this is a test Da ci d...
3966,3579,0,Never go a let you go away You belo g to me A ...
3967,752,3,Kiss me too fiercely Hold me too tight I eed ...
3968,1674,0,Why would I take you somewhere whe I already ...


In [65]:
test

,index,category,data
0,2943,0,Chorus Im just a lo g lo g way from home Im ju...
1,920,0,Im ot the best at showi g my emotio s You cut...
2,2497,3,I wish I could pai t our love These mome ts a ...
3,853,3,It feels u believable whe my bodys wrapped ar...
4,5738,3,Your touch is so magic to me The stra gest thi...
...,...,...,...
988,5986,0,Maybe you a d I ca t do great thi gs We may o...
989,2362,0,Its so sad that youve take Somethi g that I c...
990,2975,0,It bri gs big tears i to my eyes Whe I begi ...
991,4041,0,So ma y people Are o a search to fi d Withi ...


In [66]:
test_x, test_y = load_data(test)

100%|██████████| 993/993 [00:08<00:00, 111.01it/s]


In [67]:
from keras.preprocessing import sequence
from keras.utils import np_utils
# 학습 데이터 크기 일정하게 지정

# 원핫 인코딩
train_y = np_utils.to_categorical(train_y) # one_hot으로 변형
test_y = np_utils.to_categorical(test_y) # one_hot으로 변형

In [68]:
model = TFBertModel.from_pretrained("bert-base-uncased", from_pt=True)
# 토큰 인풋, 마스크 인풋, 세그먼트 인풋 정의
token_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_word_ids')
mask_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_masks')
segment_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_segment')
# 인풋이 [토큰, 마스크, 세그먼트]인 모델 정의
bert_outputs = model([token_inputs, mask_inputs, segment_inputs])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already

In [69]:
bert_outputs = bert_outputs[1]

In [70]:
# Rectified Adam 옵티마이저 사용
!pip install tensorflow_addons
import tensorflow_addons as tfa
# 총 batch size * 4 epoch = 2344 * 4
opt = tfa.optimizers.RectifiedAdam(lr=5.0e-5, total_steps = 2344*2, 
                                   warmup_proportion=0.1, min_lr=1e-5, 
                                   epsilon=1e-08, clipnorm=1.0)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [74]:
sentiment_drop = tf.keras.layers.Dropout(0.5)(bert_outputs)
sentiment_first = tf.keras.layers.Dense(4, activation='softmax', 
                                        kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.02))(sentiment_drop)
sentiment_model_2 = tf.keras.Model([token_inputs, mask_inputs, segment_inputs], sentiment_first)
sentiment_model_2.compile(optimizer=opt, loss=tf.keras.losses.CategoricalCrossentropy(), metrics = ['accuracy'])

In [75]:
sentiment_model_2.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 256)]        0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        [(None, 256)]        0                                            
__________________________________________________________________________________________________
input_segment (InputLayer)      [(None, 256)]        0                                            
__________________________________________________________________________________________________
tf_bert_model_4 (TFBertModel)   TFBaseModelOutputWit 109482240   input_word_ids[0][0]             
                                                                 input_masks[0][0]          

In [76]:
sentiment_model_2.fit(train_x, train_y, epochs=3, shuffle=True, batch_size=16
                      ,validation_data=(test_x, test_y))

Epoch 1/3
249/249 [==============================] - 559s 2s/step - loss: 0.6677 - accuracy: 0.7773 - val_loss: 0.6465 - val_accuracy: 0.7875
Epoch 2/3
249/249 [==============================] - 485s 2s/step - loss: 0.4466 - accuracy: 0.8552 - val_loss: 0.5389 - val_accuracy: 0.8207
Epoch 3/3
249/249 [==============================] - 484s 2s/step - loss: 0.2684 - accuracy: 0.9181 - val_loss: 0.6525 - val_accuracy: 0.8077


In [77]:
def predict_convert_data(data_df):
    global tokenizer
    tokens, masks, segments = [], [], []
    
    for i in tqdm(range(len(data_df))):

        token = tokenizer.encode(data_df[DATA_COLUMN][i], max_length=SEQ_LEN, truncation=True, padding='max_length')
        num_zeros = token.count(0)
        mask = [1]*(SEQ_LEN-num_zeros) + [0]*num_zeros
        segment = [0]*SEQ_LEN

        tokens.append(token)
        segments.append(segment)
        masks.append(mask)

    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)
    return [tokens, masks, segments]

# 위에 정의한 convert_data 함수를 불러오는 함수를 정의
def predict_load_data(pandas_dataframe):
    data_df = pandas_dataframe
    data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)
    data_x = predict_convert_data(data_df)
    return data_x

In [78]:
# 예측을 해봅시다!
def sentence_convert_data(data):
    global tokenizer
    tokens, masks, segments = [], [], []
    token = tokenizer.encode(data, max_length=SEQ_LEN, truncation=True, padding='max_length')
    
    num_zeros = token.count(0) 
    mask = [1]*(SEQ_LEN-num_zeros) + [0]*num_zeros 
    segment = [0]*SEQ_LEN

    tokens.append(token)
    segments.append(segment)
    masks.append(mask)

    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)
    return [tokens, masks, segments]

def movie_evaluation_predict(sentence):
    data_x = sentence_convert_data(sentence)
    predict = sentiment_model_2.predict(data_x)
    predict_value = np.ravel(predict)
    predict_answer = np.round(predict_value,0).tolist()
    
    print(predict_answer)

    if predict_answer[0] == 1.0:
      #print("슬픔가사입니다." )
      result = '슬픔'
    elif predict_answer[1] == 1.0:
      #print("행복가사입니다.")
      result = '행복'
    elif predict_answer[2] == 1.0:
      #print("편안한가사입니다.")
      result = '편안'
    elif predict_answer[3] == 1.0:
      #print("유혹가사입니다.")
      result = '유혹'
    else :
      result = '분류안됨'
    return result

In [84]:
submission = pd.read_csv('/content/dataEng.csv')

In [85]:
submission.head()

,index,Unnamed: 0,issue_date,album_name,song_name,song_gn_gnr_basket,artist_name_basket,id,lyric,lyr
0,14,14,20180720,Head In The Clouds,Red Rubies,GN1300,"88rising , Rich Brian , Yung Bans , Yung ...",245,"Aye, aye, aye Red rubies and they bleed Pop so...",Aye aye aye Red rubies and they bleed Pop some...
1,31,31,20020101,Lickin` On Both Sides,All I Want,GN1300,Mis-Teeq,409,(Rap) La la la la la (Rap) Boy I wanna be with...,Rap La la la la la Rap Boy I wanna be with you...
2,35,35,19710000,Live At Filmore West,Make It With You,GN1300,Aretha Franklin,465,Hey have you ever tried Really reaching out fo...,Hey have you ever tried Really reaching out fo...
3,46,46,20180803,Overdose (Feat. Chris Brown),Overdose (Feat. Chris Brown),GN1300,Agnez Mo,651,"Oh woah, ooh Oh, oh, eh Verse 1: Chris Brown,...",Oh woah ooh Oh oh eh Verse Chris Brown AGNEZ...
4,47,47,20140429,스타로부터 스무 발자국 OST,Desperation,GN1300,Judith Hill,669,Maybe you're where you're supposed to be Safel...,Maybe youre where youre supposed to be Safely ...


In [93]:
submission.shape

(6972, 12)

In [86]:
pred_data = submission['lyr']

In [87]:
from tqdm import tqdm as tq

In [88]:
emotion_list = []
for i in tq(range(len(pred_data))):
  emotion = movie_evaluation_predict(pred_data[i])
  emotion_list.append(emotion)

  0%|          | 2/6972 [00:00<18:19,  6.34it/s]

[0.0, 0.0, 0.0, 1.0]
[1.0, 0.0, 0.0, 0.0]


  0%|          | 4/6972 [00:00<18:56,  6.13it/s]

[1.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 1.0]


  0%|          | 6/6972 [00:00<18:31,  6.27it/s]

...

[1.0, 0.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 0.0]


100%|█████████▉| 6971/6972 [20:36<00:00,  5.20it/s]

[0.0, 1.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 1.0]


100%|██████████| 6972/6972 [20:37<00:00,  5.64it/s]

[0.0, 1.0, 0.0, 0.0]


In [89]:
emotion_list

['유혹',
 '슬픔',
 '슬픔',
 '유혹',
 '슬픔',
 '슬픔',
 '유혹',
 '슬픔',
 '유혹',
 '슬픔',
 '슬픔',
 '유혹',
 '슬픔',
 '슬픔',
 '슬픔',
 '유혹',
 '행복',
 '슬픔',
 '슬픔',
 '슬픔',
 '슬픔',
 '유혹',
 '슬픔',
 '유혹',
 '유혹',
 '유혹',
 '슬픔',
 '슬픔',
 '슬픔',
 '슬픔',
 '슬픔',
 '슬픔',
 '유혹',
 '유혹',
 '유혹',
 '유혹',
 '슬픔',
 '분류안됨',
 '유혹',
 '분류안됨',
 '유혹',
 '슬픔',
 '슬픔',
 '슬픔',
 '유혹',
 '행복',
 '슬픔',
 '슬픔',
 '슬픔',
 '슬픔',
 '행복',
 '슬픔',
 '슬픔',
 '슬픔',
 '행복',
 '슬픔',
 '유혹',
 '슬픔',
 '유혹',
 '유혹',
 '유혹',
 '슬픔',
 '분류안됨',
 '슬픔',
 '슬픔',
 '슬픔',
 '유혹',
 '슬픔',
 '유혹',
 '슬픔',
 '슬픔',
 '유혹',
 '슬픔',
 '분류안됨',
 '유혹',
 '유혹',
 '행복',
 '유혹',
 '슬픔',
 '유혹',
 '슬픔',
 '슬픔',
 '슬픔',
 '유혹',
 '행복',
 '슬픔',
 '슬픔',
 '슬픔',
 '슬픔',
 '유혹',
 '슬픔',
 '행복',
 '슬픔',
 '유혹',
 '슬픔',
 '슬픔',
 '행복',
 '슬픔',
 '슬픔',
 '슬픔',
 '유혹',
 '슬픔',
 '슬픔',
 '슬픔',
 '슬픔',
 '슬픔',
 '분류안됨',
 '행복',
 '슬픔',
 '행복',
 '유혹',
 '슬픔',
 '슬픔',
 '슬픔',
 '유혹',
 '슬픔',
 '슬픔',
 '슬픔',
 '유혹',
 '슬픔',
 '슬픔',
 '유혹',
 '유혹',
 '슬픔',
 '슬픔',
 '슬픔',
 '행복',
 '슬픔',
 '슬픔',
 '분류안됨',
 '슬픔',
 '슬픔',
 '슬픔',
 '슬픔',
 '슬픔',
 '슬픔',
 '슬픔',
 '슬픔',
 '행복',
 '슬픔',
 '유혹',
 

In [90]:
submission['emotion']=emotion_list


In [91]:
label_df = {'슬픔':0,'행복':1,'편안':2,'아찔':3}
submission["label"] = submission.loc[:,'emotion'].map(label_df)
pred_data = submission[['label','lyric']]
pred_data.columns=['category','data']

In [92]:
pred_set = predict_load_data(pred_data)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 6972/6972 [01:02<00:00, 111.08it/s]


In [94]:
preds = sentiment_model_2.predict(pred_set)

In [95]:
preds_list = preds.tolist()

In [96]:
submission['emotion_per'] = preds_list

In [97]:
sad_sum = []
happy_sum = []
sexy_sum = []
rest_sum = []

for z in tq(range(len(preds))):
  sad = round(preds_list[z][0],3)
  happy = round(preds_list[z][1],3)
  rest = round(preds_list[z][2],3)
  sexy = round(preds_list[z][3],3)
    
  sad_sum.append(sad)
  happy_sum.append(happy)
  rest_sum.append(rest)
  sexy_sum.append(sexy)

100%|██████████| 6972/6972 [00:00<00:00, 244733.26it/s]


In [98]:
submission['sad'] = sad_sum
submission['happy'] = happy_sum
submission['rest'] = rest_sum
submission['sexy'] = sexy_sum

In [99]:
submission.head()

,index,Unnamed: 0,issue_date,album_name,song_name,song_gn_gnr_basket,artist_name_basket,id,lyric,lyr,emotion,label,emotion_per,sad,happy,rest,sexy
0,14,14,20180720,Head In The Clouds,Red Rubies,GN1300,"88rising , Rich Brian , Yung Bans , Yung ...",245,"Aye, aye, aye Red rubies and they bleed Pop so...",Aye aye aye Red rubies and they bleed Pop some...,유혹,NaN,"[0.0042954059317708015, 0.003014975693076849, ...",0.004,0.003,0.003,0.989
1,31,31,20020101,Lickin` On Both Sides,All I Want,GN1300,Mis-Teeq,409,(Rap) La la la la la (Rap) Boy I wanna be with...,Rap La la la la la Rap Boy I wanna be with you...,슬픔,0.0,"[0.9805728793144226, 0.0026975038927048445, 0....",0.981,0.003,0.005,0.012
2,35,35,19710000,Live At Filmore West,Make It With You,GN1300,Aretha Franklin,465,Hey have you ever tried Really reaching out fo...,Hey have you ever tried Really reaching out fo...,슬픔,0.0,"[0.9433162808418274, 0.013903411105275154, 0.0...",0.943,0.014,0.022,0.021
3,46,46,20180803,Overdose (Feat. Chris Brown),Overdose (Feat. Chris Brown),GN1300,Agnez Mo,651,"Oh woah, ooh Oh, oh, eh Verse 1: Chris Brown,...",Oh woah ooh Oh oh eh Verse Chris Brown AGNEZ...,유혹,NaN,"[0.07708294689655304, 0.0036542483139783144, 0...",0.077,0.004,0.009,0.911
4,47,47,20140429,스타로부터 스무 발자국 OST,Desperation,GN1300,Judith Hill,669,Maybe you're where you're supposed to be Safel...,Maybe youre where youre supposed to be Safely ...,슬픔,0.0,"[0.7726805806159973, 0.04509503394365311, 0.08...",0.773,0.045,0.080,0.102


In [100]:
submission.to_csv(os.path.join(path, "submission_eng(bert).csv"), index=False)

In [ ]:
submission